# Project 4 - Mc907/Mo651 - Mobile Robotics

### Student:
Luiz Eduardo Cartolano - RA: 183012

### Instructor:
Esther Luna Colombini

### Github Link:
[Project Repository](https://github.com/luizcartolano2/mc907-mobile-robotics)

### Youtube Link:
[Link to Video](https://youtu.be/uqNeEhWo0dA)

### Subject of this Work:
The general objective of this work is to implement a deep learning approach for solve the Visual Odometry problem.

### Goals:
1. Implement and evaluate a Deep VO strategy using images from the [AirSim](https://github.com/microsoft/AirSim) simulator.

Check if azure is working! <br>
Apparently yes!

In [9]:
import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.76


In [10]:
import pandas as pd
import glob
import numpy as np
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
from torch.autograd import Function
from torch.autograd import Variable
from torchvision import models
import torch.optim as optim
import math
from scipy.spatial.transform import Rotation as R
import matplotlib
import matplotlib.pyplot as plt

## Connect to workspace

In [13]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep='\t')

deep-vo-ws	brazilsouth	docs-aml


In [16]:
ws

Workspace.create(name='deep-vo-ws', subscription_id='0deaa4ba-df61-4f72-ad8e-04805916a834', resource_group='docs-aml')

## Create an Experiment

In [14]:
from azureml.core import Experiment
experiment_name = 'deep-vo'

exp = Experiment(workspace=ws, name=experiment_name)

In [15]:
exp

Name,Workspace,Report Page,Docs Page
deep-vo,deep-vo-ws,Link to Azure Machine Learning studio,Link to Documentation


## Create or attach an existing compute target

In [19]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpucluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6 (dont work in BRAZIL)
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                                min_nodes=compute_min_nodes,
                                                                max_nodes=compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(
        ws, compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout.
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20)

    # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-11-28T16:03:42.153000+00:00', 'errors': None, 'creationTime': '2019-11-28T16:03:05.078445+00:00', 'modifiedTime': '2019-11-28T16:03:50.933458+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Create DataSet on Azure

In [28]:
from azureml.core.datastore import Datastore
from azureml.core.dataset import Dataset

In [30]:
# retrieve an existing datastore in the workspace by name
datastore = Datastore.get(ws, 'deep-vo-dataset')

ErrorResponseException: (UserError) Could not find datastore: deep-vo-dataset.

## Create a directory

In [20]:
script_folder = os.path.join(os.getcwd(), "deep-vo")
os.makedirs(script_folder, exist_ok=True)

## Create an estimator

In [21]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment('deep_vo')
cd = CondaDependencies.create(pip_packages=['azureml-sdk','scikit-learn','azureml-dataprep[pandas,fuse]>=1.1.14', 
                                            'opencv-python', 'torch', 'torchvision', 'scipy'])
env.python.conda_dependencies = cd

In [25]:
from azureml.train.sklearn import SKLearn

est = SKLearn(source_directory=script_folder,
              compute_target=compute_target,
              environment_definition=env, 
              entry_script='main.py')

WARNING - If environment_definition or conda_dependencies_file_path is specified, Azure ML will not install any framework related packages on behalf of the user.
WARNING - This compute target type doesn't support non-Docker runs; overriding run configuration enable Docker.
WARNING - You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['scikit-learn', 'scipy']. We cannot guarantee image build will succeed.


In [26]:
run = exp.submit(config=est)
run

Submitting /Users/luizeduardocartolano/OneDrive/DUDU/Unicamp/IC/MC907/workspace/deep-vo directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


TrainingException: TrainingException:
	Message: ====================================================================

While attempting to take snapshot of /Users/luizeduardocartolano/OneDrive/DUDU/Unicamp/IC/MC907/workspace/deep-vo
Your project exceeds the file limit of 2000.

====================================================================


	InnerException SnapshotException:
	Message: ====================================================================

While attempting to take snapshot of /Users/luizeduardocartolano/OneDrive/DUDU/Unicamp/IC/MC907/workspace/deep-vo
Your project exceeds the file limit of 2000.

====================================================================


	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "====================================================================\n\nWhile attempting to take snapshot of /Users/luizeduardocartolano/OneDrive/DUDU/Unicamp/IC/MC907/workspace/deep-vo\nYour project exceeds the file limit of 2000.